# Introdução
Os datasets de 'matriculas', organizados por período acadêmico, contêm dados relacionados as turmas e a situação (aprovado, reprovado, trancado, etc) dos discentes dessas turmas. Esses dados nos são interessantes na análise das avaliações dos docentes, já que podem ser relacionados com as mesmas de várias maneiras. Contudo, do modo que estão organizados agora, esses dados não se "encaixam" facilmente no nosso dataset principal (avaliação dos docentes), então precisamos limpar e modificar esses datasets para que seus dados possam ser úteis em nossa análise. Para isso, vamos agrupar os dados por turma (coluna 'id_turma'), afim de relacionar com a turma em nosso dataset principal, e condensar dados úteis que seriam perdidos nessa agrupagem em novas colunas (quantidade de aprovados, quantidade de reprovados, média geral da turma, etc).

O presente notebook é um exemplo do processo feito para cada um desses datasets de 'matriculas', afim de deixar claro o que foi feito na tratagem desses dados.

# Preparando o ambiente

Importando bibliotecas

In [1]:
import pandas as pd
import numpy as np

O dataset que utilizaremos nesse exemplo é o último dataset de matriculas disponível (referente a 2022.2)

In [19]:
dfMatriculas = pd.read_csv("../CSVs/Baixados/matriculas-2022.2.csv", sep=";")

# Entendendo o dataset

In [3]:
dfMatriculas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 658565 entries, 0 to 658564
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id_turma             658565 non-null  int64  
 1   discente             658565 non-null  object 
 2   id_curso             656766 non-null  float64
 3   unidade              595449 non-null  float64
 4   nota                 578285 non-null  object 
 5   reposicao            595449 non-null  object 
 6   faltas_unidade       595449 non-null  float64
 7   media_final          594452 non-null  object 
 8   numero_total_faltas  612310 non-null  float64
 9   descricao            658565 non-null  object 
dtypes: float64(4), int64(1), object(5)
memory usage: 50.2+ MB


O dataset é composto por 10 colunas, e tem 658565 linhas. Alguns atributos (colunas) estão com tipo 'object' quando eram para estar com tipo 'float', como 'nota' e 'media_final'. Iremos consertar isso futuramente.

In [4]:
print(f"Quantidade de turmas: {dfMatriculas['id_turma'].unique().size}")
print(f"Quantidade de valores nulos por coluna: ")
print(f"{dfMatriculas.isnull().sum()}")

Quantidade de turmas: 8212
Quantidade de valores nulos por coluna: 
id_turma                   0
discente                   0
id_curso                1799
unidade                63116
nota                   80280
reposicao              63116
faltas_unidade         63116
media_final            64113
numero_total_faltas    46255
descricao                  0
dtype: int64


Ao todo, existem 8212 turmas no dataset, contudo muitas dessas turmas podem não ser turmas "de verdade", futuramente vamos analisar melhor isso.
Dentre esses valores nulos, muitos serão descartados pois não nos são interessantes, enquanto outros serão mantidos. Por exemplo, um aluno que trancou uma disciplina aparecerá com média final nula, e essa é uma informação interessante para nós.

In [5]:
print(f"Valores únicos de 'descricao': {dfMatriculas["descricao"].unique()}")

Valores únicos de 'descricao': ['APROVADO' 'DESISTENCIA' 'REPROVADO POR MÉDIA E POR FALTAS' 'INDEFERIDO'
 'TRANCADO' 'APROVADO POR NOTA' 'REPROVADO' 'CANCELADO'
 'REPROVADO POR FALTAS' 'REPROVADO POR NOTA E FALTA' 'REPROVADO POR NOTA'
 'EXCLUIDA' 'CUMPRIU' 'DISPENSADO' 'EM ESPERA' 'AGUARDANDO DEFERIMENTO'
 'MATRICULADO']


A nós, interessa apenas os valores de 'descricao' que dizem respeito a situação final de um aluno naquela turma (aprovado, reprovado, etc). Muitos desses valores únicos não nos serão úteis, então serão descartados.

# Limpando o dataset

Vamos começar removendo as ocorrências de 'descricao' que não nos serão úteis (já especificados).

In [6]:
# Lista dos valores indesejados na coluna 'descricao',
valores_indesejados = ['INDEFERIDO', 'EXCLUIDA', 'CUMPRIU', 'DISPENSADO', 'EM ESPERA', 'AGUARDANDO DEFERIMENTO', 'MATRICULADO', 'CANCELADO']

# Filtrar o DataFrame, mantendo apenas linhas onde 'descricao' NÃO está na lista
dfMatriculas_filtro1 = dfMatriculas[~dfMatriculas['descricao'].isin(valores_indesejados)]

# Checando se o filtro foi aplicado corretamente
print(f"Valores únicos de 'descricao' pós filtro: {dfMatriculas_filtro1["descricao"].unique()}")

Valores únicos de 'descricao' pós filtro: ['APROVADO' 'DESISTENCIA' 'REPROVADO POR MÉDIA E POR FALTAS' 'TRANCADO'
 'APROVADO POR NOTA' 'REPROVADO' 'REPROVADO POR FALTAS'
 'REPROVADO POR NOTA E FALTA' 'REPROVADO POR NOTA']


Vamos checar como isso afetou a contagem de nulos no dataset

In [7]:
# Calcula nulos antes e depois
nulos_antes = dfMatriculas.isna().sum()
nulos_depois = dfMatriculas_filtro1.isna().sum()

# DataFrame de comparação
comparacao_nulos = pd.DataFrame({
    'Nulos_Antes': nulos_antes,
    'Nulos_Depois': nulos_depois,
    'Diferença': nulos_antes - nulos_depois
})

print("=== Comparação de Valores Nulos antes e pós filtro 1 ===")
print(comparacao_nulos)

=== Comparação de Valores Nulos antes e pós filtro 1 ===
                     Nulos_Antes  Nulos_Depois  Diferença
id_turma                       0             0          0
discente                       0             0          0
id_curso                    1799          1681        118
unidade                    63116         30305      32811
nota                       80280         38136      42144
reposicao                  63116         30305      32811
faltas_unidade             63116         30305      32811
media_final                64113         20305      43808
numero_total_faltas        46255         10917      35338
descricao                      0             0          0


Alguns alunos possuem o valor em 'unidade' nulo, mas com 'descricao' = 'aprovado'. Julgamos que essas podem não ser turmas de verdade, então vamos remover essas ocorrências.

In [8]:
# Ocorrências de 'unidade' = null/NaN e 'descricao' = 'APROVADO'
df_null_e_aprovado = (dfMatriculas_filtro1[
    (dfMatriculas_filtro1['unidade'].isna()) & 
    (dfMatriculas_filtro1['descricao'] == 'APROVADO')
])

print(f"Quantidade de ocorrências de 'unidade' nula  e aluno aprovado: {df_null_e_aprovado.size}")
print(f"Exemplos de ocorrências de 'unidade' nula  e aluno aprovado:")
print(f"{df_null_e_aprovado.head(5)}")

print(f"\n ===================================================================== \n")

# Filtrando o dataset para quando NÃO há as ocorrências especificadas
dfMatriculas_filtro2 = (dfMatriculas_filtro1[
    ~(
        (dfMatriculas_filtro1['unidade'].isna()) & 
        (dfMatriculas_filtro1['descricao'] == 'APROVADO')
    )
])

# Checando se a filtragem foi feita corretamente
df_null_e_aprovado = (dfMatriculas_filtro2[
    (dfMatriculas_filtro2['unidade'].isna()) & 
    (dfMatriculas_filtro2['descricao'] == 'APROVADO')
])

print(f"Quantidade de ocorrências de 'unidade' nula  e aluno aprovado pós filtro 2: {df_null_e_aprovado.size}")
print(f"Diferença de tamanho do dataset antes e pós filtro 2: {dfMatriculas_filtro1.size - dfMatriculas_filtro2.size}")

Quantidade de ocorrências de 'unidade' nula  e aluno aprovado: 211680
Exemplos de ocorrências de 'unidade' nula  e aluno aprovado:
      id_turma                          discente  id_curso  unidade nota  \
3981  57709858  3201c5a41680f1be3fd705282435cfb1  315735.0      NaN  NaN   
3982  57709858  03ce9d83adf2a2714da477cb62ecfbcc  315736.0      NaN  NaN   
3983  57709858  246083838651b1815f41581c1fb4a2f8  315735.0      NaN  NaN   
3984  57709858  f27d4b5121d169e8ce5636ad9a8aad65  315736.0      NaN  NaN   
3985  57709858  0ea411c2f095648992df53774f125be0  315735.0      NaN  NaN   

     reposicao  faltas_unidade media_final  numero_total_faltas descricao  
3981       NaN             NaN         5,0                  0.0  APROVADO  
3982       NaN             NaN         4,0                 10.0  APROVADO  
3983       NaN             NaN         5,0                  0.0  APROVADO  
3984       NaN             NaN         4,0                 10.0  APROVADO  
3985       NaN             NaN  

Mais uma vez, vamos checar como isso afetou a quantidade de valores nulos

In [9]:
# Calcula nulos antes e depois
nulos_antes = dfMatriculas_filtro1.isna().sum()
nulos_depois = dfMatriculas_filtro2.isna().sum()

# DataFrame de comparação
comparacao_nulos = pd.DataFrame({
    'Nulos_Antes': nulos_antes,
    'Nulos_Depois': nulos_depois,
    'Diferença': nulos_antes - nulos_depois
})

print("=== Comparação de Valores Nulos antes e pós filtro 2 ===")
print(comparacao_nulos)

=== Comparação de Valores Nulos antes e pós filtro 2 ===
                     Nulos_Antes  Nulos_Depois  Diferença
id_turma                       0             0          0
discente                       0             0          0
id_curso                    1681          1138        543
unidade                    30305          9137      21168
nota                       38136         16968      21168
reposicao                  30305          9137      21168
faltas_unidade             30305          9137      21168
media_final                20305         17294       3011
numero_total_faltas        10917         10905         12
descricao                      0             0          0


Como já foi dito antes, alguns nulos em 'media_final' são esperados. Mas existem casos em que nulos nessa coluna não são esperados, especificamente quando 'descricao' é 'APROVADO' ou 'APROVADO POR NOTA', uma vez que para ser aprovado o aluno deve ter alguma nota média final, ou quando 'descricao' é 'REPROVADO POR NOTA', já que precisa de uma nota para ser reprovado por nota. Vamos explorar esses casos.

In [10]:
# Valores únicos de 'descricao' onde 'media_final' é null/NaN
valores_unicos_media_nula = (
    dfMatriculas_filtro2[
        (dfMatriculas_filtro2['media_final'].isna())
    ]
)['descricao'].unique()

print(f"Valores de 'descricao' que ainda possuem ocorrências de 'media_final' nula: {valores_unicos_media_nula}")

Valores de 'descricao' que ainda possuem ocorrências de 'media_final' nula: ['DESISTENCIA' 'TRANCADO' 'APROVADO' 'REPROVADO' 'REPROVADO POR FALTAS']


'DESISTENCIA', 'TRANCADO', 'REPROVADO' e 'REPROVADO POR FALTAS' são situações em que se é possível ter, dessa forma a única situação em que se ainda tem ocorrências inesperadas de 'media_final' nula é de alunos com 'APROVADO'. Vamos checar explorar essas ocorrências e remover as turmas em que isso acontece.

In [11]:
# Ocorrências de 'media_final' = null/NaN e 'descricao' = 'APROVADO'
df_media_nula_e_aprovado = dfMatriculas_filtro2[(
    (dfMatriculas_filtro2['media_final'].isna()) &
    (dfMatriculas_filtro2['descricao'] == 'APROVADO')
)]

print(f"Quantidade de ocorrências de média final nula e aluno aprovado: {df_media_nula_e_aprovado.size}")

# Valores únicos de 'id_turma' onde 'media_final' = null/NaN e 'descricao' = 'APROVADO'
turmas_indesejadas = df_media_nula_e_aprovado['id_turma'].unique()
print(f"Quantidade de turmas em que existem alunos com média final nula e aluno aprovado: {turmas_indesejadas.size}")

# Filtrando o dataset para quando 'id_turma' NÃO está nas turmas indesejadas
dfMatriculas_filtro3 = dfMatriculas_filtro2[~dfMatriculas_filtro2['id_turma'].isin(turmas_indesejadas)]

# Checando se a filtragem foi feita corretamente
df_media_nula_e_aprovado = dfMatriculas_filtro3[(
    (dfMatriculas_filtro3['media_final'].isna()) &
    (dfMatriculas_filtro3['descricao'] == 'APROVADO')
)]

print(f"Quantidade de ocorrências de 'unidade' nula  e aluno aprovado pós filtro 3: {df_null_e_aprovado.size}")
print(f"Diferença de quantidade de turmas no dataset antes e pós filtro 3: {
        dfMatriculas_filtro2['id_turma'].unique().size - dfMatriculas_filtro3['id_turma'].unique().size
    }")

Quantidade de ocorrências de média final nula e aluno aprovado: 15000
Quantidade de turmas em que existem alunos com média final nula e aluno aprovado: 63
Quantidade de ocorrências de 'unidade' nula  e aluno aprovado pós filtro 3: 0
Diferença de quantidade de turmas no dataset antes e pós filtro 3: 63


Vamos checar uma última vez como isso afetou a quantidade de nulos

In [12]:
# Calcula nulos antes e depois
nulos_antes = dfMatriculas_filtro2.isna().sum()
nulos_depois = dfMatriculas_filtro3.isna().sum()

# DataFrame de comparação
comparacao_nulos = pd.DataFrame({
    'Nulos_Antes': nulos_antes,
    'Nulos_Depois': nulos_depois,
    'Diferença': nulos_antes - nulos_depois
})

print("=== Comparação de Valores Nulos antes e pós filtro 3 ===")
print(comparacao_nulos)

=== Comparação de Valores Nulos antes e pós filtro 3 ===
                     Nulos_Antes  Nulos_Depois  Diferença
id_turma                       0             0          0
discente                       0             0          0
id_curso                    1138          1136          2
unidade                     9137          9103         34
nota                       16968         15360       1608
reposicao                   9137          9103         34
faltas_unidade              9137          9103         34
media_final                17294         15686       1608
numero_total_faltas        10905         10871         34
descricao                      0             0          0


Por fim, precisamos tratar o fato de que devido as diferentes 'unidade', um mesmo aluno pode aparecer mais de uma vez dentro de uma mesma turma, isso é indesejado pois quando formos calcular a quantidade de aprovados (ou etc) dentro de uma turma, um mesmo aluno será contado mais de uma vez. Para tratar isso basta remover duplicatas de um mesmo aluno numa mesma turma, os outros dados que nos interessam (media final e descrição) não serão afetados uma vez que eles são iguais para todas as ocorrências do aluno.

In [13]:
# Remover duplicatas, mantendo apenas a primeira ocorrência de cada aluno por turma
dfMatriculas_filtro4 = dfMatriculas_filtro3.drop_duplicates(subset=['id_turma', 'discente'], keep='first')

# Verificar resultado
print(f"Total de registros antes da remoção: {len(dfMatriculas_filtro3)}")
print(f"Total de registros após remoção: {len(dfMatriculas_filtro4)}")
print(f"Total de turmas antes da remoção: {len(dfMatriculas_filtro3['id_turma'].unique())}")
print(f"Total de turmas após remoção: {len(dfMatriculas_filtro4['id_turma'].unique())}")

Total de registros antes da remoção: 588680
Total de registros após remoção: 145787
Total de turmas antes da remoção: 6831
Total de turmas após remoção: 6831


Nenhuma turma foi removida, enquanto diversas linhas foram removidas, o que é o resultado esperado.

# Construindo o novo dataset

Com o dataset original limpo, podemos construir o novo dataset. As colunas presentes no novo dataset serão a quantidade de cada categoria de 'descricao', o id da turma e uma média da média final.

In [14]:
print(f"Categorias (valores únicos) de 'descricao': {dfMatriculas_filtro4['descricao'].unique()}")

Categorias (valores únicos) de 'descricao': ['APROVADO' 'DESISTENCIA' 'REPROVADO POR MÉDIA E POR FALTAS' 'TRANCADO'
 'APROVADO POR NOTA' 'REPROVADO' 'REPROVADO POR FALTAS'
 'REPROVADO POR NOTA E FALTA' 'REPROVADO POR NOTA']


Esse novo dataset terá as colunas: id_turma; media_final_geral; qtd_aprovados; qtd_desistencias; qtd_reprovados_por_media_e_faltas; qtd_trancados; qtd_aprovados_por_nota; qtd_reprovados; qtd_reprovados_por_faltas; qtd_reprovados_por_nota.

In [15]:
categorias = [
    'APROVADO', 'DESISTENCIA', 'REPROVADO POR MÉDIA E POR FALTAS',
    'TRANCADO', 'APROVADO POR NOTA', 'REPROVADO',
    'REPROVADO POR FALTAS', 'REPROVADO POR NOTA E FALTA',
    'REPROVADO POR NOTA'
]


Antes de criar o novo dataset, precisamos corrigir o problema da coluna 'media_final' com o tipo 'object', convertendo essa linha para float.

In [16]:
# Função para converter os dados para float
def converter_para_float(valor):
    try:
        if pd.isna(valor):
            return np.nan
        if isinstance(valor, str):
            return float(valor.replace(',', '.'))
        return float(valor)
    except (ValueError, TypeError):
        return np.nan

# Criando uma cópia do dataframe para evitar warnings
dfMatriculas_filtro4Copia = dfMatriculas_filtro4.copy()

# Aplicando a função de conversão no dataset
dfMatriculas_filtro4Copia['media_final'] = dfMatriculas_filtro4Copia['media_final'].apply(converter_para_float)

# Checando se a conversão foi feita com sucesso
dfMatriculas_filtro4Copia.info()

<class 'pandas.core.frame.DataFrame'>
Index: 145787 entries, 0 to 658562
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id_turma             145787 non-null  int64  
 1   discente             145787 non-null  object 
 2   id_curso             145478 non-null  float64
 3   unidade              138942 non-null  float64
 4   nota                 137374 non-null  object 
 5   reposicao            138942 non-null  object 
 6   faltas_unidade       138942 non-null  float64
 7   media_final          137227 non-null  float64
 8   numero_total_faltas  138673 non-null  float64
 9   descricao            145787 non-null  object 
dtypes: float64(5), int64(1), object(4)
memory usage: 12.2+ MB


Com a coluna 'media_final' no tipo correto, agora podemos gerar o novo dataset.

In [34]:
# Agrupar por turma e calcular média
dfMatriculasAgrupado = dfMatriculas_filtro4Copia.groupby('id_turma').agg(
    media_final_geral=('media_final', 'mean')
)

# Calcular as quantidades de cada categoria e criar novas colunas com esses valores
for categoria in categorias:
    col_name = f'qtd_{categoria.lower().replace(" ", "_")}'
    df_temp = (
        dfMatriculas_filtro4Copia[dfMatriculas_filtro4Copia['descricao'] == categoria]
        .groupby('id_turma')
        .size()
        .rename(col_name)
    )
    dfMatriculasAgrupado = dfMatriculasAgrupado.join(df_temp, how='left')

# Convertendo tipo e preenchendo nulos com 0 nas colunas criadas
dfMatriculasAgrupado = dfMatriculasAgrupado.fillna(0).astype({col: int for col in dfMatriculasAgrupado.columns if col.startswith('qtd_')})
dfMatriculasAgrupado = dfMatriculasAgrupado.reset_index()

dfMatriculasAgrupado.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6831 entries, 0 to 6830
Data columns (total 11 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   id_turma                              6831 non-null   int64  
 1   media_final_geral                     6831 non-null   float64
 2   qtd_aprovado                          6831 non-null   int64  
 3   qtd_desistencia                       6831 non-null   int64  
 4   qtd_reprovado_por_média_e_por_faltas  6831 non-null   int64  
 5   qtd_trancado                          6831 non-null   int64  
 6   qtd_aprovado_por_nota                 6831 non-null   int64  
 7   qtd_reprovado                         6831 non-null   int64  
 8   qtd_reprovado_por_faltas              6831 non-null   int64  
 9   qtd_reprovado_por_nota_e_falta        6831 non-null   int64  
 10  qtd_reprovado_por_nota                6831 non-null   int64  
dtypes: float64(1), in

# Gerando um CSV com o novo dataset

In [20]:
dfMatriculasAgrupado.to_csv(
    '../CSVs/Gerados/exemplo_turmas-2022.2_agrupado.csv',  # Nome do arquivo de saída
    index=False,                # Não incluir o índice no arquivo
    sep=';',                    # Separador (padrão é vírgula)
    header=True,                # Incluir cabeçalho
)

# Fazendo uma função para o tratamento

Vamos fazer uma função com todo o tratamento, para usar na automação futuramente.

In [43]:
def processar_matriculas(ano, periodo):
    # Configurações iniciais
    categorias_descricao = [
        'APROVADO', 'DESISTENCIA', 'REPROVADO POR MÉDIA E POR FALTAS',
        'TRANCADO', 'APROVADO POR NOTA', 'REPROVADO',
        'REPROVADO POR FALTAS', 'REPROVADO POR NOTA E FALTA',
        'REPROVADO POR NOTA'
    ]
    
    try:
        # Ler o arquivo original
        df = pd.read_csv(f"../CSVs/Baixados/matriculas-{ano}.{periodo}.csv", sep=';', dtype={'id_turma': str})

        # Primeiro filtro - Remover categorias indesejadas
        valores_indesejados = ['INDEFERIDO', 'EXCLUIDA', 'CUMPRIU', 'DISPENSADO',
                             'EM ESPERA', 'AGUARDANDO DEFERIMENTO', 'MATRICULADO', 'CANCELADO']
        df_filtrado = df[~df['descricao'].isin(valores_indesejados)].copy()

        # Segundo filtro - Remover unidade nula + aprovados
        mascara = ~((df_filtrado['unidade'].isna()) & (df_filtrado['descricao'] == 'APROVADO'))
        df_filtrado = df_filtrado[mascara]

        # Terceiro filtro - Remover turmas com média nula e aprovados
        turmas_problematicas = df_filtrado[
            (df_filtrado['media_final'].isna()) & 
            (df_filtrado['descricao'] == 'APROVADO')
        ]['id_turma'].unique()
        
        df_filtrado = df_filtrado[~df_filtrado['id_turma'].isin(turmas_problematicas)]

        # Quarto filtro - Remover duplicatas
        df_filtrado = df_filtrado.drop_duplicates(subset=['id_turma', 'discente'], keep='first')

        # Converter media_final para float
        def converter_media(valor):
            try:
                if pd.isna(valor): return np.nan
                return float(str(valor).replace(',', '.'))
            except: return np.nan
            
        df_filtrado.loc[:, 'media_final'] = df_filtrado['media_final'].apply(converter_media)

        # Agrupar dados
        df_agrupado = df_filtrado.groupby('id_turma').agg(
            media_final_geral=('media_final', 'mean')
        )

        # Adicionar contagem por categoria
        for categoria in categorias_descricao:
            col_name = f'qtd_{categoria.lower().replace(" ", "_")}'
            temp_df = (
                df_filtrado[df_filtrado['descricao'] == categoria]
                .groupby('id_turma')
                .size()
                .rename(col_name)
            )
            df_agrupado = df_agrupado.join(temp_df, how='left')

        # Tratar valores nulos e tipos
        df_agrupado = df_agrupado.fillna(0).astype(
            {col: int for col in df_agrupado.columns if col.startswith('qtd_')}
        )

        df_agrupado = df_agrupado.reset_index()


        return df_agrupado # para comparar

    except Exception as e:
        print(f"Erro ao processar {ano}.{periodo}: {str(e)}")

Agora vamos testar a função para o ano de 2022 e periodo 2, e comparar o resultado com nosso dataset gerado anteriormente.

In [44]:
dfMatriculasFuncao = processar_matriculas(2022, 2)

C:\Users\pedro\AppData\Local\Temp\ipykernel_11592\1359543812.py:60: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_agrupado = df_agrupado.fillna(0).astype(


In [45]:
print(f"{dfMatriculasFuncao.head(5)}")

   id_turma  media_final_geral  qtd_aprovado  qtd_desistencia  \
0  57702272                2.0             0                0   
1  57702312                0.0             0                1   
2  57702322                0.0             0                1   
3  57702332                0.0             0                1   
4  57702333                0.0             0                1   

   qtd_reprovado_por_média_e_por_faltas  qtd_trancado  qtd_aprovado_por_nota  \
0                                     0             0                      0   
1                                     0             0                      0   
2                                     0             0                      0   
3                                     0             1                      0   
4                                     0             1                      0   

   qtd_reprovado  qtd_reprovado_por_faltas  qtd_reprovado_por_nota_e_falta  \
0              1                         0        

In [46]:
print(f"{dfMatriculasAgrupado.head(5)}")

   id_turma  media_final_geral  qtd_aprovado  qtd_desistencia  \
0  57702272                2.0             0                0   
1  57702312                0.0             0                1   
2  57702322                0.0             0                1   
3  57702332                0.0             0                1   
4  57702333                0.0             0                1   

   qtd_reprovado_por_média_e_por_faltas  qtd_trancado  qtd_aprovado_por_nota  \
0                                     0             0                      0   
1                                     0             0                      0   
2                                     0             0                      0   
3                                     0             1                      0   
4                                     0             1                      0   

   qtd_reprovado  qtd_reprovado_por_faltas  qtd_reprovado_por_nota_e_falta  \
0              1                         0        